In [1]:
import numpy  as np
import pandas as pd

import matplotlib.pyplot as plt

import remap_values      as rv
import df_visualizations as dv

In [6]:
# price_doc new sale price

In [37]:
train_df = pd.read_csv( 'inp_data/train.csv' )
test_df  = pd.read_csv( 'inp_data/test.csv'  )

target_price = np.log10( train_df       ['price_doc'] .copy() )
train_df     =           train_df.drop( ['price_doc'] , axis=1 )

big_df = train_df.append( test_df ).copy()

In [3]:
print train_df.shape
print test_df.shape

(30471, 291)
(7662, 291)


In [38]:
# Combine some values
big_df['N_build']    = big_df[ ['build_count_wood','build_count_frame','build_count_mix',
                                'build_count_slag','build_count_foam','build_count_block',
                                'build_count_panel','build_count_monolith','build_count_brick'] ].sum( axis=1 )

big_df['Good_build'] = (big_df['build_count_wood'] + big_df['build_count_frame'] + big_df['build_count_mix'] + 
                        big_df['build_count_slag'] + big_df['build_count_foam']) / big_df['N_build']

big_df['Bad_build' ] = (big_df['build_count_block']+ big_df['build_count_panel'] + big_df['build_count_monolith'] + 
                        big_df['build_count_brick'] ) / big_df['N_build']

big_df['Good_build'] = big_df['Good_build'].fillna( big_df['Good_build'].median() )
big_df['Bad_build' ] = big_df['Bad_build' ].fillna( big_df['Bad_build' ].median() )

big_df.ix[ big_df['Good_build'] > 1.0 , 'Good_build' ] = big_df['Good_build'].median()
big_df.ix[ big_df['Bad_build' ] > 1.0 , 'Bad_build'  ] = big_df['Bad_build' ].median()




# Do the above replace for these, make list of those prob
power_plants = ['thermal_power_plant_raion','nuclear_reactor_raion']
ruin_list    = ['incineration_raion','oil_chemistry_raion','radiation_raion']
misc_list    = ['culture_objects_top_25','big_market_raion','detention_facility_raion','big_road1_1line']

big_list = power_plants+ruin_list+misc_list

for item in big_list:
    big_df[item] = big_df[item].replace( {'no':0,'yes':1} ).fillna(0)
    
big_df['big_market_raion'     ] = big_df['big_market_raion'     ].fillna(0)
big_df['nuclear_reactor_raion'] = big_df['nuclear_reactor_raion'].fillna(0) 
    
# Do some data cleaning    
big_df['product_type'] = big_df['product_type'].replace( {'Investment':0, 'OwnerOccupier':1} ).fillna(0)
big_df['state'       ] = big_df['state'       ].replace( {33.:3.}).fillna(0.)
big_df['material'    ] = big_df['material'    ].replace( {3:0})
big_df['material'    ] = big_df['material'    ].replace( {6:3}).fillna(0)

big_df['floor'    ] = big_df['floor'    ].fillna( 0 )
big_df['state'    ] = big_df['state'    ].fillna( 0 )
big_df['max_floor'] = big_df['max_floor'].fillna( 0 )


# Fill nulls with medians
for adj in ['prom_part_5000','life_sq','num_room','kitch_sq']:
    big_df[adj] = big_df[adj].fillna( big_df[adj].median() )

    
    
# Fill build year oddballs, far outside range-fill in median
# Otherwise, use material to estimate a year
# Still outlying? Toss in the new mean
big_df   [                              'build_year' ] = big_df['build_year'].replace( {2.00520090e+07:2007} )
big_df.ix[ big_df['build_year'] < 1000, 'build_year' ] = big_df['build_year'].mean()
big_df.ix[ big_df['build_year'] > 2020, 'build_year' ] = big_df['build_year'].mean()

for floor in big_df['max_floor'].unique():
    mean = big_df.ix[ big_df['max_floor'] == floor, 'build_year' ].mean()
    big_df.ix[ big_df['max_floor'] == floor, 'build_year' ] = big_df.ix[ big_df['max_floor'] == floor, 'build_year' ].fillna( mean )
big_df['build_year'] = big_df['build_year'].fillna( big_df['build_year'].mean() )




# Put floors in range
for flr in ['floor','max_floor']:
    big_df.ix[ (big_df[flr] >  2) & (big_df[flr] <=  5), flr ] = 3
    big_df.ix[ (big_df[flr] >  5) & (big_df[flr] <= 10), flr ] = 4
    big_df.ix[ (big_df[flr] > 10) & (big_df[flr] <= 30), flr ] = 5
    big_df.ix[ (big_df[flr] > 30) & (big_df[flr] <= 50), flr ] = 6
    big_df.ix[ (big_df[flr] > 50)                      , flr ] = 7
big_df.ix[ big_df['floor'] > big_df['max_floor'], 'max_floor' ] = big_df.ix[ big_df['floor'] > big_df['max_floor'], 'floor'] 

big_df['sub_area'] = rv.numerize_col( big_df, 'sub_area' )



# Take the material, and break it into binary classification
foo = rv.binary_classification( big_df[ ['material'] ] )
for col in foo.columns.values:
    big_df[col] = foo[col]
big_df = big_df.drop( 'material', axis = 1 )


# Take the ecology, set up binary classifiction
big_df['ecology'] = big_df['ecology'].replace( {'no data':0, 'poor':1, 'satisfactory':2, 'good':3, 'excellent':4} )
foo = rv.binary_classification( big_df[ ['ecology'] ] )
for col in foo.columns.values:
    big_df[col] = foo[col]
big_df = big_df.drop( 'ecology', axis = 1 )

In [39]:
# This stuff had near 0 correlation to final price
drop_list_age      = ['0_6_all','0_6_male','0_6_female','7_14_all','7_14_male','7_14_female','0_17_all','0_17_male','0_17_female','16_29_all','16_29_male','16_29_female','0_13_all','0_13_male','0_13_female']
drop_list_aGroup   = ['young_all', 'young_male', 'young_female', 'work_all', 'work_male', 'work_female', 'ekder_all', 'ekder_male', 'ekder_female']
drop_list_pop      = ['full_all','male_f','female_f']
drop_list_neigh    = ['raion_popul']
drop_list_green    = ['green_part_3000','green_part_2000','green_part_1500','green_part_1000','green_part_500']
drop_list_school   = ['preschool_km','school_education_centers_top_20_raion','school_quota','preschool_quota','school_quota','children_preschool','preschool_education_centers_raion']
drop_list_build    = ['build_count_wood','build_count_frame','build_count_mix','build_count_slag','build_count_foam','build_count_block','build_count_panel','build_count_monolith','build_count_brick','build_count_before_1920','build_count_1921-1945','build_count_1946-1970','build_count_1971-1995','build_count_after_1995']
drop_list_auto     = ['metro_km_avto','metro_min_walk','metro_km_walk','ID_metro']
drop_list_railroad = ['railroad_station_avto_km', 'railroad_station_walk_min', 'railroad_station_walk_km', 'ID_railroad_station_avto', 'ID_railroad_station_walk', 'railroad_1line','railroad_terminal_raion','ID_railroad_terminal'] 
drop_list_metro    = ['ID_metro', 'metro_km_avto', 'metro_min_walk', 'metro_km_walk']
drop_list_sport    = ['sport_objects_raion', 'public_transport_station_min_walk', 'sport_count_500', 'sport_count_1000', 'sport_count_1500', 'sport_count_2000', 'sport_count_3000']
drop_list_office   = ['office_raion',  'office_count_500', 'office_count_5000', 'office_sqm_500', 'office_count_1000', 'office_sqm_1000', 'office_count_1500', 'office_sqm_1500', 'office_count_2000', 'office_sqm_2000', 'office_count_3000', 'office_sqm_3000' ]


drop_list_misc   = ['hospital_beds_raion','university_top_20_raion','public_transport_station_min_walk','additional_education_raion',
                    'culture_objects_top_25_raion','shopping_centers_raion','water_1line','ID_bus_terminal','big_road1_1line',
                    'green_zone_part', 'indust_part', 'green_part_5000', 'prom_part_5000', 'metro_min_avto', 
                    'railroad_station_avto_min', 'office_sqm_5000','trc_sqm_5000', 'Bad_build']

drop_list_power  = ['thermal_power_plant_raion']
drop_list_ruin   = ['incineration_raion','oil_chemistry_raion','radiation_raion']
drop_list_market = ['market_count_500','market_count_1000','market_count_1500','market_count_2000','market_count_3000']
drop_list_info   = ['raion_build_count_with_material_info','raion_build_count_with_builddate_info']
drop_list_road   = ['ID_big_road1', 'ID_big_road2']
drop_list_indus  = ['prom_part_500', 'prom_part_1000', 'prom_part_1500', 'prom_part_2000', 'prom_part_3000']
drop_list_mall   = ['trc_count_500', 'trc_sqm_500', 'trc_count_1000', 'trc_sqm_1000', 'trc_count_1500', 'trc_sqm_1500', 'trc_count_2000', 'trc_sqm_2000', 'trc_count_3000', 'trc_sqm_3000']

drop_list_church     = ['church_count_500', 'church_count_1000', 'church_count_1500', 'church_count_2000', 'church_count_3000']
drop_list_mosque     = ['mosque_count_500', 'mosque_count_1000', 'mosque_count_1500', 'mosque_count_2000', 'mosque_count_3000']
drop_list_leisure    = ['leisure_count_500', 'leisure_count_1000', 'leisure_count_1500', 'leisure_count_2000', 'leisure_count_3000']
drop_list_big_church = ['big_church_count_500', 'big_church_count_1000', 'big_church_count_1500', 'big_church_count_2000', 'big_church_count_3000']


super_list = drop_list_age    + drop_list_aGroup  + drop_list_pop        + drop_list_neigh    + drop_list_green  + \
             drop_list_school + drop_list_build   + drop_list_auto       + drop_list_railroad + drop_list_metro  + \
             drop_list_sport  + drop_list_office  + drop_list_power      + drop_list_ruin     + drop_list_market + \
             drop_list_info   + drop_list_road    + drop_list_indus      + drop_list_mall     + drop_list_church + \
             drop_list_mosque + drop_list_leisure + drop_list_big_church + \
             drop_list_misc

big_df = big_df.drop( super_list, axis=1 )

# First to do:
foo = big_df['cafe_count_5000'           ].copy()
bar = big_df['cafe_count_5000_price_high'].copy()
# Drop everyhing else
big_df = big_df.drop( big_df.columns[ big_df.columns.str.contains('cafe') ], axis=1 )
big_df['cafe_count_5000']            = foo.copy()
big_df['cafe_count_5000_price_high'] = bar.copy()

In [40]:
foo = big_df.ix[:, big_df.columns.str.contains('raion') ].copy()
foo['healthcare_centers_raion'      ] = rv.normalize_column_sigma( foo, 'healthcare_centers_raion'      , lower_bound = False )
foo['school_education_centers_raion'] = rv.normalize_column_sigma( foo, 'school_education_centers_raion', lower_bound = False )

rain_pca  = rv.generate_reduced_PCA(    foo, 1, contain_str='raion', corr_df=target_price )
km_pca    = rv.generate_reduced_PCA( big_df, 1, contain_str='_km'  , corr_df=target_price, sigma_normalize=True, lower_bound = False, col_names='km' )
count_pca = rv.generate_reduced_PCA( big_df, 1, contain_str='count', corr_df=target_price, sigma_normalize=True )

big_df = big_df.drop( big_df.columns.values[ big_df.columns.str.contains('raion') ], axis=1 )
big_df = big_df.drop( big_df.columns.values[ big_df.columns.str.contains(  '_km') ], axis=1 )
big_df = big_df.drop( big_df.columns.values[ big_df.columns.str.contains('count') ], axis=1 )

big_df[  rain_pca.columns.values[0] ] =  rain_pca
big_df[    km_pca.columns.values[0] ] =    km_pca
big_df[ count_pca.columns.values[0] ] = count_pca

In [41]:
# log the area
big_df['area_m'] = np.log10( big_df['area_m'] )

big_df.ix[ big_df['N_build' ] >  800, 'N_build' ] =  800
big_df.ix[ big_df['full_sq' ] >  700, 'full_sq' ] =  700
big_df.ix[ big_df['life_sq' ] >  100, 'life_sq' ] =  100
big_df.ix[ big_df['kitch_sq'] >   20, 'kitch_sq'] =   20
big_df.ix[ big_df['num_room'] >    6, 'num_room'] =    6

log_list         = [ 'full_sq','life_sq','kitch_sq','N_build','children_school']
norm_sigma_upper = [ 'area_m' ]

for col in log_list:
    big_df[col] = np.log10( big_df[col] + 1 )
    big_df[col] = rv.scale_column_sigma( big_df, col, n_sigma=3.0 )
    
for col in norm_sigma_upper:
    big_df[col] = rv.scale_column_sigma( big_df, col, n_sigma=3.0 )

In [42]:
ordered_list = ['id', 'timestamp', 'full_sq', 'life_sq', 'floor', 'max_floor', 'build_year', 'num_room', 'kitch_sq', 'product_type',
                'state', 'area_m', 'children_school', 'culture_objects_top_25', 'N_build', 'Good_build', 
                'raion_pca_0', 'km_pca_0', 'count_pca_0',
                'material_0.0', 'material_1.0', 'material_2.0', 'material_3.0', 'material_4.0', 'material_5.0',
                'ecology_0', 'ecology_1', 'ecology_2', 'ecology_3', 'ecology_4']

binarize_list = ['state','sub_area']

foo = big_df[binarize_list].copy()
bar = rv.binary_classification( foo, ignore='state', upperLim=200 )

big_df = big_df[ordered_list]

for i in range( 0, 146 ):
    col = 'sub_area_'+str(i)    
    big_df[col] = bar[col]

In [43]:
big_df.columns.values

array(['id', 'timestamp', 'full_sq', 'life_sq', 'floor', 'max_floor',
       'build_year', 'num_room', 'kitch_sq', 'product_type', 'state',
       'area_m', 'children_school', 'culture_objects_top_25', 'N_build',
       'Good_build', 'raion_pca_0', 'km_pca_0', 'count_pca_0',
       'material_0.0', 'material_1.0', 'material_2.0', 'material_3.0',
       'material_4.0', 'material_5.0', 'ecology_0', 'ecology_1',
       'ecology_2', 'ecology_3', 'ecology_4', 'sub_area_0', 'sub_area_1',
       'sub_area_2', 'sub_area_3', 'sub_area_4', 'sub_area_5',
       'sub_area_6', 'sub_area_7', 'sub_area_8', 'sub_area_9',
       'sub_area_10', 'sub_area_11', 'sub_area_12', 'sub_area_13',
       'sub_area_14', 'sub_area_15', 'sub_area_16', 'sub_area_17',
       'sub_area_18', 'sub_area_19', 'sub_area_20', 'sub_area_21',
       'sub_area_22', 'sub_area_23', 'sub_area_24', 'sub_area_25',
       'sub_area_26', 'sub_area_27', 'sub_area_28', 'sub_area_29',
       'sub_area_30', 'sub_area_31', 'sub_area_32', '

In [15]:
'''
timestamp: date of transaction
full_sq: total area in square meters, including loggias, balconies and other non-residential areas
life_sq: living area in square meters, excluding loggias, balconies and other non-residential areas
floor: for apartments, floor of the building
max_floor: number of floors in the building
material: wall material
build_year: year built
num_room: number of living rooms
kitch_sq: kitchen area
state: apartment condition
product_type: owner-occupier purchase or investment
'''

In [16]:
'''
raion=sub_area
sub_area: name of the district
full_all: subarea population
male_f, female_f: subarea population by gender
young_*: population younger than working age
work_*: working-age population
ekder_*: retirement-age population
n_m_{all|male|female}: population between n and m years old
build_count_*: buildings in the subarea by construction type or year
x_count_500: the number of x within 500m of the property
x_part_500: the share of x within 500m of the property
_sqm_: square meters
cafe_count_d_price_p: number of cafes within d meters of the property that have an average bill under p RUB
trc_: shopping malls
prom_: industrial zones
green_: green zones
metro_: subway
_avto_: distances by car
mkad_: Moscow Circle Auto Road
ttk_: Third Transport Ring
sadovoe_: Garden Ring
bulvar_ring_: Boulevard Ring
kremlin_: City center
zd_vokzaly_: Train station
oil_chemistry_: Dirty industry
ts_: Power plant
'''

'\nsub_area: name of the district\nfull_all: subarea population\nmale_f, female_f: subarea population by gender\nyoung_*: population younger than working age\nwork_*: working-age population\nekder_*: retirement-age population\nn_m_{all|male|female}: population between n and m years old\nbuild_count_*: buildings in the subarea by construction type or year\nx_count_500: the number of x within 500m of the property\nx_part_500: the share of x within 500m of the property\n_sqm_: square meters\ncafe_count_d_price_p: number of cafes within d meters of the property that have an average bill under p RUB\ntrc_: shopping malls\nprom_: industrial zones\ngreen_: green zones\nmetro_: subway\n_avto_: distances by car\nmkad_: Moscow Circle Auto Road\nttk_: Third Transport Ring\nsadovoe_: Garden Ring\nbulvar_ring_: Boulevard Ring\nkremlin_: City center\nzd_vokzaly_: Train station\noil_chemistry_: Dirty industry\nts_: Power plant\n'